In [1]:
#프로젝트의 루트 디렉토리를 설정하기 위한 코드
import os
current_dir = os.getcwd()  # 현재 작업 디렉토리
print(os.path.dirname(current_dir))
# print(os.path.dirname(os.path.dirname(current_dir)))
# 루트 경로 설정 (예: 프로젝트 루트 경로)
project_root = os.path.dirname(current_dir)
os.chdir(project_root)  # 루트 경로로 변경

c:\sb-fsts\sb-fsts


In [2]:
from app.utils.crud_sql import SQLExecutor
from app.utils.database import get_db, get_db_session

sql_executor = SQLExecutor()

In [3]:
query = """
        SELECT 
            *
        FROM 
            fsts.test_table
    """

params = {}

with get_db_session() as db:
    result = sql_executor.execute_select(db, query, params)

print(result)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbe in position 97: invalid start byte

In [4]:
# RETURNING * 꼭 붙여야 함.
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

id = 'id4'
name = 'name4'

params = {
    "id": id,
    "name": name
}

with get_db_session() as db:
    result = sql_executor.execute_insert(db, query, params)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbe in position 97: invalid start byte

In [5]:
#여러 데이터를 한번에 삽입하고 싶을 때
# 여러 데이터 정의
data = [
    {"id": "id5", "name": "name5"},
    {"id": "id6", "name": "name6"},
    {"id": "id7", "name": "name7"}
]

# 다중 데이터 삽입 쿼리
query = """
    INSERT INTO fsts.test_table
    (id, name)
    VALUES (:id, :name)
    RETURNING *;
"""

# 데이터 삽입 및 결과 저장
results = []
with get_db_session() as db:
    for row in data:
        result = sql_executor.execute_insert(db, query, row)
        results.append(result)

print("삽입된 데이터:")
print(results)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbe in position 97: invalid start byte

In [6]:
# UPDATE 테스트
query_update = """
    UPDATE fsts.test_table
    SET name = :name
    WHERE id = :id
    RETURNING *;
"""
#:는 파라미터 바인딩 형식

params_update = {
    "id": "id2",
    "name": "HONG"
}

with get_db_session() as db:
    result_update = sql_executor.execute_update(db, query_update, params_update)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xbe in position 97: invalid start byte

# DynamoDB

In [3]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute, ListAttribute, MapAttribute
from datetime import datetime
import time

# 매매 이력 (trading_history)에서 사용될 속성 정의
class TradeHistory(MapAttribute):
    trading_logic = UnicodeAttribute()
    symbol = UnicodeAttribute()
    symbol_name = UnicodeAttribute()
    position = UnicodeAttribute()
    price = NumberAttribute()
    quantity = NumberAttribute()
    created_at = NumberAttribute()
    updated_at = UnicodeAttribute(null=True)
    data_type = UnicodeAttribute()

# DynamoDB 테이블 모델 정의
class TradingBot(Model):
    class Meta:
        table_name = "fsts-trading-bot"
        region = "ap-northeast-2"

    trading_bot_name = UnicodeAttribute(hash_key=True)  # PK 설정
    created_at = NumberAttribute()
    trading_history = ListAttribute(of=TradeHistory)  # 리스트 저장 가능

# 테이블 생성 (최초 1회 실행)
# if not TradingBot.exists():
#     TradingBot.create_table(read_capacity_units=5, write_capacity_units=5, wait=True)

In [4]:
from pynamodb.models import Model
from pynamodb.attributes import UnicodeAttribute, NumberAttribute
import time
import uuid

class TradingHistory(Model):
    class Meta:
        table_name = "fsts-trading-history"
        region = "ap-northeast-2"

    trading_bot_name = UnicodeAttribute(hash_key=True)  # ✅ PK
    created_at = NumberAttribute(range_key=True)  # ✅ SK (밀리세컨드 단위)
    updated_at = UnicodeAttribute(null=True)
    symbol = UnicodeAttribute()
    position = UnicodeAttribute()
    price = NumberAttribute()
    quantity = NumberAttribute()
    data_type = UnicodeAttribute()

In [7]:
from pynamodb.transactions import TransactWrite
from pynamodb.connection import Connection
from botocore.exceptions import ClientError

def add_trade(trading_bot_name, symbol, position, price, quantity, data_type):
    max_retries = 3  # 최대 재시도 횟수
    retry_count = 0

    while retry_count < max_retries:
        try:
            created_at = int(time.time() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

            new_trade = TradingHistory(
                trading_bot_name=trading_bot_name,
                created_at=created_at,
                updated_at=None,
                symbol=symbol,
                position=position,
                price=price,
                quantity=quantity,
                data_type=data_type
            )

            connection = Connection()

            with TransactWrite(connection=connection) as transaction:
                transaction.save(new_trade, condition=(TradingHistory.created_at.does_not_exist()))
                print(f"✅ 트랜잭션 성공: {created_at}")
                return True  # 성공적으로 저장되면 종료

        except ClientError as e:
            if e.response["Error"]["Code"] == "TransactionCanceledException":
                print("❌ 중복된 created_at 감지! 새로운 값으로 재시도...")
                retry_count += 1
            else:
                raise  # 다른 에러 발생 시 예외 던지기

    print("🚨 최대 재시도 횟수 초과! 거래 저장 실패")
    return False

In [5]:
def execute_save(data_model):
        max_retries = 3  # 최대 재시도 횟수
        retry_count = 0

        while retry_count < max_retries:
            try:
                connection = Connection()

                created_at = int(time.time() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

                with TransactWrite(connection=connection) as transaction:
                    # transaction.save(data_model, condition=(TradingHistory.created_at.does_not_exist()))
                    result = transaction.save(data_model, condition=(type(data_model).created_at.does_not_exist()))
                    print(f"✅ 트랜잭션 성공: {created_at}, {result}")
                    return True  # 성공적으로 저장되면 종료

            except ClientError as e:
                if e.response["Error"]["Code"] == "TransactionCanceledException":
                    print("❌ 중복된 created_at 감지! 새로운 값으로 재시도...")
                    retry_count += 1
                else:
                    raise  # 다른 에러 발생 시 예외 던지기

        print("🚨 최대 재시도 횟수 초과! 거래 저장 실패")
        return False

In [7]:
from app.utils.dynamodb.model.user_info_model import UserInfo
from pytz import timezone

import json

kst = timezone("Asia/Seoul")
current_time = datetime.now(kst)
created_at = int(current_time.timestamp() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성


file_path = "./dashboard_web/trading_logic.json"
with open(file_path, "r", encoding="utf-8") as file:
    trading_logic = json.load(file)

# 사용 예시
available_buy_logic = trading_logic["available_buy_logic"]
available_sell_logic = trading_logic["available_sell_logic"]

print(list(available_buy_logic.values()))
print(list(available_sell_logic.values()))

# trading_logic 리스트 설정
buy_trading_logic = list(available_buy_logic.values())
sell_trading_logic = list(available_sell_logic.values())
    
model = UserInfo(
    name='schedulerbot',
    created_at=created_at,
    updated_at=None,
    id='schedulerbot',
    password='password3',
    app_key='PSoDCgOHYDScFNsCWkpWbkqYzFz5S7VSOW64',
    secret_key='V0R0YRyHchOlseMP/snQbYKZOQtKXqs41m+gzBHV7sMtisaKX/2VRIM/n8uppyWY1TDZ8zT+56S0AtxjSusDWCGal7+u4a7dRD2yjGO8lrbbMxDjzRPJZi1n1wVejLm4dU6QP6OO6jiD+LB9nP4daiPt0R/B2Q0llkIfHKk6GONQu5pdarc=',
    kis_id='@2526582',
    account='64927922',
    buy_trading_logic = buy_trading_logic,
    sell_trading_logic = sell_trading_logic
)
execute_save(model)

['rsi_trading', 'penetrating', 'engulfing', 'engulfing2', 'counterattack', 'harami', 'doji_star', 'morning_star', 'check_wick', 'stochastic_trading', 'macd_trading', 'mfi_trading', 'rsi+mfi', 'ema_breakout_trading', 'bollinger_band_trading', 'bollinger+ema', 'ema_breakout_trading2', 'trend_entry_trading', 'bottom_rebound_trading', 'sma_breakout_trading']
['rsi_trading', 'dark_cloud', 'down_engulfing', 'down_engulfing2', 'down_counterattack', 'down_harami', 'down_doji_star', 'evening_star', 'check_wick', 'stochastic_trading', 'macd_trading', 'mfi_trading', 'rsi+mfi', 'bollinger_band_trading', 'top_reversal_sell_trading', 'downtrend_sell_trading']
✅ 트랜잭션 성공: 1743567639372, None


True

In [ ]:
from app.utils.dynamodb.model.stock_symbol_model import StockSymbol
from pytz import timezone


kst = timezone("Asia/Seoul")
# 현재 시간을 KST로 변환

for i in kospi:
    current_time = datetime.now(kst)
    created_at = int(current_time.timestamp() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

    model = StockSymbol(
        symbol=i['symbol'],
        created_at=created_at,
        updated_at=None,
        symbol_name=i['name'],
        type='kospi200'
    )
    execute_save(model)

In [ ]:
from app.utils.dynamodb.model.user_info_model import UserInfo
from pytz import timezone

kst = timezone("Asia/Seoul")
current_time = datetime.now(kst)
created_at = int(current_time.timestamp() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

model = UserInfo(
    name = 'weeklybot',
    created_at=created_at,
    updated_at=None,
    id='weeklybot',
    password='password4',
    app_key='PSoDCgOHYDScFNsCWkpWbkqYzFz5S7VSOW64',
    secret_key='V0R0YRyHchOlseMP/snQbYKZOQtKXqs41m+gzBHV7sMtisaKX/2VRIM/n8uppyWY1TDZ8zT+56S0AtxjSusDWCGal7+u4a7dRD2yjGO8lrbbMxDjzRPJZi1n1wVejLm4dU6QP6OO6jiD+LB9nP4daiPt0R/B2Q0llkIfHKk6GONQu5pdarc=',
    kis_id='@2526582',
    account='64927922',
    virtual_kis_id ='@2526582',
    virtual_app_key = 'PSNmcwmbyfnvVM4yTDw3F1wBS07mjPBc6vvy',
    virtual_secret_key = 'Xe1y6dFH7y0s4sQ6JMG8d8UsSZlNlfnNpCRM526xS6DvQ+k9BQyvXQWNQ0fWARDVxk20oSf77dDRZPmPVVTAMTpf5+ZR7fnK5qU3OlwP6Ft4YD/qazFgCsHK8C3Fywb0A1r83nDPeXbDbMnuzPVF2YGZsA0dtGBL3h7SBK+RMGg72QlPdto=',
    virtual_account = '50131876',
    max_allocation = 0.01,
    target_trade_value_krw = 1000000
    
)
execute_save(model)

✅ 트랜잭션 성공: 1743997873091, None


TransactWriteError: Failed to write transaction items

### 데이터 조회

In [9]:
# 테이블 전체 탐색
history = TradingHistory.scan()

for trade in history:
    print(f"- Symbol: {trade.symbol}, Price: {trade.price}, Quantity: {trade.quantity}, Created At: {trade.created_at}")

- Symbol: 021240, Price: 84800, Quantity: 1, Created At: 1741136449356
- Symbol: 111770, Price: 46700, Quantity: 1, Created At: 1741136466446
- Symbol: 457190, Price: 53300, Quantity: 1, Created At: 1741136481226
- Symbol: 024110, Price: 15380, Quantity: 1, Created At: 1741136599864
- Symbol: 042660, Price: 81700, Quantity: 1, Created At: 1741136602197
- Symbol: 021240, Price: 85000, Quantity: 1, Created At: 1741136608555
- Symbol: 111770, Price: 46850, Quantity: 1, Created At: 1741136626361
- Symbol: 457190, Price: 53100, Quantity: 1, Created At: 1741136640231
- Symbol: 021240, Price: 81500, Quantity: 1, Created At: 1741155018273
- Symbol: 128940, Price: 262000, Quantity: 1, Created At: 1741155026458
- Symbol: 111770, Price: 46800, Quantity: 1, Created At: 1741155035952
- Symbol: 457190, Price: 53000, Quantity: 1, Created At: 1741155051100
- Symbol: 012330, Price: 256000, Quantity: 1, Created At: 1741501861302
- Symbol: 111770, Price: 46550, Quantity: 1, Created At: 1741501861903
- Sy

In [52]:
# 특정 trading_bot_name의 데이터 조회
history = TradingHistory.query("MyTradingBot_1234")

for trade in history:
    print(f"- Symbol: {trade.symbol}, Price: {trade.price}, Quantity: {trade.quantity}, Created At: {trade.created_at}")

- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740407407897
- Symbol: AAPL, Price: 200, Quantity: 1, Created At: 1740408172439


In [ ]:
kospi200_items = StockSymbol.scan(
    filter_condition=(StockSymbol.type == 'kospi200')
)

In [8]:
# 특정 trading_bot_name의 데이터 조회
history = UserInfo.query("schedulerbot")

for trade in history:
    print(f"- buy_trading_logic: {trade.buy_trading_logic}")

- buy_trading_logic: ['rsi_trading', 'penetrating', 'engulfing', 'engulfing2', 'counterattack', 'harami', 'doji_star', 'morning_star', 'check_wick', 'stochastic_trading', 'macd_trading', 'mfi_trading', 'rsi+mfi', 'ema_breakout_trading', 'bollinger_band_trading', 'bollinger+ema', 'ema_breakout_trading2', 'trend_entry_trading', 'bottom_rebound_trading', 'sma_breakout_trading']


In [ ]:
for i in kospi200_items:
    print(i.symbol)

데이터 삭제

In [ ]:
from app.utils.dynamodb.model.stock_symbol_model import StockSymbol

# 먼저 전체 스캔해서 type이 "kospi200"인 항목만 필터링
for item in StockSymbol.scan(StockSymbol.type == "kospi200"):
    print(f"Deleting: {item.symbol}, type: {item.type}")
    item.delete() #데이터 삭제

Deleting: 073240, type: kospi200
Deleting: 336260, type: kospi200
Deleting: 012330, type: kospi200
Deleting: 111770, type: kospi200
Deleting: 326030, type: kospi200
Deleting: 204320, type: kospi200
Deleting: 007310, type: kospi200
Deleting: 008770, type: kospi200
Deleting: 032830, type: kospi200
Deleting: 086790, type: kospi200
Deleting: 017800, type: kospi200
Deleting: 047040, type: kospi200
Deleting: 006360, type: kospi200
Deleting: 005380, type: kospi200
Deleting: 005930, type: kospi200
Deleting: 023530, type: kospi200
Deleting: 009420, type: kospi200
Deleting: 001450, type: kospi200
Deleting: 034020, type: kospi200
Deleting: 003490, type: kospi200
Deleting: 207940, type: kospi200
Deleting: 028260, type: kospi200
Deleting: 009540, type: kospi200
Deleting: 028670, type: kospi200
Deleting: 015760, type: kospi200
Deleting: 030000, type: kospi200
Deleting: 010140, type: kospi200
Deleting: 402340, type: kospi200
Deleting: 012450, type: kospi200
Deleting: 030200, type: kospi200
Deleting: 

In [ ]:
import requests
import pandas as pd
import json
# 크롤링 코드

# 요청할 URL
url = "http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd"

headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "keep-alive",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "http://data.krx.co.kr",
    "Referer": "http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201010101&idxIndMidclssCd=02&money=1&share=1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36",
    "X-Requested-With": "XMLHttpRequest"
}

payload = {
    "bld": "dbms/MDC/STAT/standard/MDCSTAT00601",
    "locale": "ko_KR",
    "tboxindIdx_finder_equidx0_1": "코스피 200",
    "indIdx": "1",
    "indIdx2": "028",
    "codeNmindIdx_finder_equidx0_1": "코스피 200",
    "param1indIdx_finder_equidx0_1": "",
    "trdDd": "20250402",  # 조회일자
    "money": "3",
    "csvxls_isNo": "false"
}

# POST 요청 보내기
response = requests.post(url, headers=headers, data=payload)

# 응답 상태 확인 및 데이터 파싱
if response.status_code == 200:
    try:
        data = response.json()
        # KRX의 응답에서 데이터를 담은 키는 보통 "OutBlock_1"입니다.
        if "output" in data:
            items = data["output"]
            print(f"총 종목 개수: {len(items)}")
            result = []
            for item in items:
                # 종목명과 종목코드에 해당하는 키는 응답에 따라 달라질 수 있습니다.
                name = item.get("ISU_ABBRV") or item.get("종목명")
                symbol = item.get("ISU_SRT_CD") or item.get("종목코드")
                if name and symbol:
                    result.append({"name": name, "symbol": symbol})
            # JSON 형태로 출력 (한글이 깨지지 않도록 ensure_ascii=False)
            print(json.dumps(result, ensure_ascii=False, indent=2))
        else:
            print("응답 데이터에 'output' 키가 없습니다:", data)
    except Exception as e:
        print("JSON 파싱 중 오류 발생:", e)
else:
    print("요청 실패, 상태 코드:", response.status_code)


In [ ]:
from app.utils.dynamodb.model.stock_symbol_model import StockSymbol
from pytz import timezone


kst = timezone("Asia/Seoul")
# 현재 시간을 KST로 변환

for i in result:
    current_time = datetime.now(kst)
    created_at = int(current_time.timestamp() * 1000)  # ✅ 밀리세컨드 단위로 SK 생성

    model = StockSymbol(
        symbol=i['symbol'],
        created_at=created_at,
        updated_at=None,
        symbol_name=i['name'],
        type='kospi200'
    )
    execute_save(model)

In [2]:
#테이블 생성 코드
from app.utils.dynamodb.model.auto_trading_balance_model import AutoTradingBalance 

if not AutoTradingBalance.exists():
    AutoTradingBalance.create_table(read_capacity_units=1, write_capacity_units=1, wait=True)
    print("✅ DynamoDB 테이블 생성 완료")
else:
    print("ℹ️ 테이블이 이미 존재합니다.")

✅ DynamoDB 테이블 생성 완료


In [3]:
from app.utils.dynamodb.model.auto_trading_model import AutoTrading
from app.utils.dynamodb.model.auto_trading_balance_model import AutoTradingBalance
from app.utils.auto_trading_bot import AutoTradingBot
from pytz import timezone

trading_bot_name = 'bnuazz15bot'

auto_trading_stock = AutoTradingBot(id='bnuazz15', virtual = True)

auto_trading_stock._upsert_account_balance(trading_bot_name)



모의투자 API 객체가 성공적으로 생성되었습니다.
[04/04 16:27:39] WARNING: API 호출 횟수를 초과하여 호출 유량 획득까지 대기합니다.
✅ 트랜잭션 성공: 1743751660067
[잔고 저장] 025900
✅ 트랜잭션 성공: 1743751660730
[잔고 저장] 025980
✅ 트랜잭션 성공: 1743751661380
[잔고 저장] 114090
✅ 트랜잭션 성공: 1743751662052
[잔고 저장] 141080
✅ 트랜잭션 성공: 1743751662664
[잔고 저장] 225570
✅ 트랜잭션 성공: 1743751663312
[잔고 저장] 241710
✅ 트랜잭션 성공: 1743751663969
[잔고 저장] 263750
✅ 트랜잭션 성공: 1743751664605
[잔고 저장] 326030
